In [ ]:
import pandas as pd
import dlib
from tqdm import tqdm
import os
import itertools
import shutil
from pathlib import Path
from PIL import Image, ImageDraw
import face_recognition
from perturb_filters.glasses.put_glasses import apply_glasses

In [ ]:
LABELS = ['female', 'male']

In [ ]:
def apply_makeup(parent_folder, image_fn ,output_fn):
    """
    Applies make up filter on a single image and saves it to a given directory.
    
    parent_folder : str
        Parent folder of image
    image_fn : str
        Filename of image
    output_fn : str
        Output image to save
    """
    # Load the jpg file into a numpy array
    image = face_recognition.load_image_file(parent_folder+image_fn)

    # Find all facial features in all the faces in the image
    face_landmarks_list = face_recognition.face_landmarks(image)

    pil_image = Image.fromarray(image)
    for face_landmarks in face_landmarks_list:
        d = ImageDraw.Draw(pil_image, 'RGBA')

        # Make the eyebrows into a nightmare
        d.polygon(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 128))
        d.polygon(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 128))
        d.line(face_landmarks['left_eyebrow'], fill=(68, 54, 39, 150), width=5)
        d.line(face_landmarks['right_eyebrow'], fill=(68, 54, 39, 150), width=5)

        # Gloss the lips
        d.polygon(face_landmarks['top_lip'], fill=(150, 0, 0, 128))
        d.polygon(face_landmarks['bottom_lip'], fill=(150, 0, 0, 128))
        d.line(face_landmarks['top_lip'], fill=(150, 0, 0, 64), width=8)
        d.line(face_landmarks['bottom_lip'], fill=(150, 0, 0, 64), width=8)

        # Sparkle the eyes
        d.polygon(face_landmarks['left_eye'], fill=(255, 255, 255, 30))
        d.polygon(face_landmarks['right_eye'], fill=(255, 255, 255, 30))

        # Apply some eyeliner
        d.line(face_landmarks['left_eye'] + [face_landmarks['left_eye'][0]], fill=(0, 0, 0, 110), width=6)
        d.line(face_landmarks['right_eye'] + [face_landmarks['right_eye'][0]], fill=(0, 0, 0, 110), width=6)
        fn = Path(output_fn + image_fn)
        if not fn.is_dir():
            pil_image.save(output_fn + image_fn)    # Change this to male or female

In [ ]:
def gen_test_makeup():
    """
    Generates makeup test datasets for each CV set
    
    """
    original = 'cv_datasets/'
    folder = os.listdir(original)
    for set_fn in folder:
        for gender in ['female']:
            ori_path = original + set_fn + '/test/' + gender + '/'
            temp = os.listdir(ori_path)
            for i in tqdm(range(len(temp)), "Generating makeup for {} images...".format(gender)):
                image_fn = temp[i]
                target = original + set_fn + "/test_"
                target_makeup = target+'makeup/'+ gender + '/'

                # Generate make up dataset
                    # Create new folder if doesn't already exist
                Path(target_makeup).mkdir(parents=True, exist_ok=True)    
                # Save image in this folder
                    # If doesn't already exist
                if not Path(target_makeup + image_fn).is_dir():
                    apply_makeup(ori_path, image_fn, target_makeup)

In [ ]:
# original = 'cv_datasets/'
# folder = os.listdir(original)
# for set_fn in folder:
#     for gender in LABELS:
#         ori_path = original + set_fn + '/test/' + gender + '/'
#         temp = os.listdir(ori_path)
#         for i in tqdm(range(len(temp)), "Generating glasses for {} images...".format(gender)):
#             image_fn = temp[i]
#             target = original + set_fn + "/test_"
#             target_glasses = target+'glasses/'+ gender + '/'

#             # Generate make up dataset
#                 # Create new folder if doesn't already exist
#             Path(target_glasses).mkdir(parents=True, exist_ok=True)    
#             # Save image in this folder
#                 # If doesn't already exist
#             if not Path(target_glasses + image_fn).is_dir():
#                 apply_glasses(ori_path, image_fn, target_glasses)

In [ ]:
def gen_glasses_dataset(gender):
    """
    Generates glasses dataset for a particular gender
    """
    target = f'glasses_{gender}/'
    root = f'{os.getcwd()}/combined_filtered/full'
    ori_path = f'{root}/{gender}'
    folder = os.listdir(ori_path)
    for i in tqdm(range(len(folder)), f"Generating glasses for all {gender} images..."):
        image_fn = folder[i]
        target_glasses = target
        if not Path(target_glasses + image_fn).is_dir():
            apply_glasses(ori_path, image_fn, target_glasses)
            
gen_glasses_dataset("female")

In [ ]:
def gen_makeup_dataset(gender):
    """
    Generates make up dataset for a particular gender
    """
    target = f'makeup_{gender}/'
    root = f'/home/monash/Desktop/fyp-work/fyp-ma-13/fyp-models/preprocessing/combined_filtered/full'
    ori_path = f'{root}/{gender}/'
    folder = os.listdir(ori_path)
    for i in tqdm(range(len(folder)), f"Generating make up for all {gender} images..."):
        image_fn = folder[i]
        target_makeup = target
        # Generate make up dataset
            # Create new folder if doesn't already exist
        Path(target_makeup).mkdir(parents=True, exist_ok=True)   

        if not Path(target_makeup + image_fn).is_dir():
            apply_makeup(ori_path, image_fn, target_makeup)
            
gen_makeup_dataset("male")